In [79]:
# Hacking a non-streaming CNN Model

import cv2

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))
# Any results you write to the current directory are saved as output. 
#KS Ask about this line

import matplotlib.pyplot as plt
%matplotlib inline 
from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data
import bson
import io
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder

import time
import threading
import itertools
from tqdm import *

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Lambda
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical


import tensorflow as tf

In [55]:
# Looking specifically over the range for the following categories, while aready numeric
# I will use label encoder to take out any sense of ordinal value 

targets = [1000003437, 1000003407, 1000003400, 1000003404, 1000003402]

In [56]:
label_encoder = LabelEncoder()
label_encoder.fit(targets)

LabelEncoder()

In [57]:
data = bson.decode_file_iter(open('/Volumes/G-DRIVE mobile/Image_Data_DSI/train.bson', 'rb'))

In [58]:
# load data and convert it into an iterable version of the data

In [59]:
it = iter(data)

In [60]:
# This is the method to stream through the test set and pull out the data 
X = []
y = []
counter=0
with tqdm(total=6009411) as pbar:
    for i in range(6009411):
        piece = next(it)
        if piece.get('category_id') in targets:
            counter = counter + 1
            if counter % 100 == 0: 
                print(counter)
            img = piece.get('imgs')[0]
            pic = io.BytesIO(img.get('picture', None))
            im = imread(pic)
            im = im.astype('float32') / 255.0
            X.append(im)
            np.array(X)

            category = piece.get('category_id', "")
            label = label_encoder.transform([category])
            y.append(label)
            np.array(y)
            pbar.update()





  0%|          | 101/6009411 [00:02<75:24:20, 22.14it/s]

100


  0%|          | 200/6009411 [00:25<515:43:15,  3.24it/s]

200


  0%|          | 300/6009411 [00:49<334:55:53,  4.98it/s]

...

  0%|          | 6499/6009411 [2:30:21<3905:37:09,  2.34s/it]

6500


  0%|          | 6599/6009411 [2:34:33<4270:03:16,  2.56s/it]

6600


  0%|          | 6660/6009411 [2:37:15<5774:10:01,  3.46s/it]


In [61]:
len(X)


6660

In [73]:
X =np.array(X)

In [74]:
type(X)

numpy.ndarray

In [75]:
y = np.array(y)

In [76]:
type(y)

numpy.ndarray

In [ ]:
# Making these into arrays may have created a problem. Can try to convert again
# or open new notebook and run again without the array line, hopefully will be faster. 


In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2003)

X_train = X_train.reshape(X_train.shape[0], 180, 180, 3)
X_test = X_test.reshape(X_test.shape[0], 180, 180, 3)


In [78]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(180, 180, 3), activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(16, (4,4), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2)

NameError: name 'MaxPool2D' is not defined

In [45]:
# Try 2 will be shrinking down to this size

In [43]:
#def test_data(iterable):
#     lst_test = []
#     for i in range(82):
#         lst_test.append(next(it))

In [44]:
# def pull_100k(iterable):
#     lst_items = []
#     for i in range(100000):
#         lst_items.append(next(iterable))

In [ ]:
# def pull_20k(iterable):
#     lst_20_items = []
#     for i in range(20000):
#         lst_20_items.append(next(iterable))

In [50]:
#data = bson.decode_file_iter(open('train_example.bson', 'rb'))